In [1]:
from google.cloud import bigquery
import requests, datetime
import pandas as pd

CREDS = r'C:\Users\chiahaowang\Documents\PythonCode\bison-prod-bqapi-tool.json'
client = bigquery.Client.from_service_account_json(json_credentials_path = CREDS)

##### SessionActive

In [ ]:
query_sessionactive = \
"""
select ActiveDate as LoginDate, UserID as transferUID, count(UserID) as LoginTimes
from `igs-bison-prod.ExtData_Bison.SessionActive`
where ActiveDate >= '2024-08-01' and ActiveDate <= '2024-09-30'
group by transferUID,LoginDate
"""

job1 = client.query(query_sessionactive)
df = job1.to_dataframe()
# df.to_csv(r'C:\Users\chiahaowang\Documents\PythonCode\LossData\02_raw_data\07_depositRecords\depositRecords.csv', index = False)

##### AccountCreate

In [ ]:
query_accountcreate = \
"""
select distinct B.*
from
(
  select ActiveDate as LoginDate, UserID as transferUID, count(UserID) as LoginTimes
  from `igs-bison-prod.ExtData_Bison.SessionActive`
  where ActiveDate >= '2024-09-01' and ActiveDate <= '2024-09-30'
  group by transferUID,LoginDate
) as A
left join
(
  select UserID as transferUID, AccountCreateDate as CreateDate
  , case when Channel = 'App Store' then 1 
         when Channel = 'Google Play' then 2
    end as Channel
  , case when OpenType = 'ios' then 10 
         when OpenType = 'android' then 11
    end as OpenType
  from `igs-bison-prod.ExtData_Bison.AccountCreate`
  where AccountCreateDate >= '2022-12-15'
) as B on A.transferUID = B.transferUID
"""

job1 = client.query(query_accountcreate)
df = job1.to_dataframe()
# df.to_csv(r'C:\Users\chiahaowang\Documents\PythonCode\LossData\02_raw_data\07_depositRecords\depositRecords.csv', index = False)

##### SeeeisonLength

In [ ]:
query_sessionlength = \
"""
select UserID as transferUID, '' as GameID, datetime(timestamp_seconds(cast(PlayStartTs/1000000 as INT64)), 'Asia/Taipei') as StartTime, datetime(timestamp_seconds(cast(PlayEndTs/1000000 as INT64)), 'Asia/Taipei') as EndTime
, cast(round((PlayEndTs-PlayStartTs)/1000000/60, 0) as INT64) as Min
from `igs-bison-prod.ExtData_Bison.SessionLength`
where PlayDate >= '2024-08-01' and PlayDate <= '2024-09-30'
"""

job1 = client.query(query_sessionlength)
df = job1.to_dataframe()
# df.to_csv(r'C:\Users\chiahaowang\Documents\PythonCode\LossData\02_raw_data\07_depositRecords\depositRecords.csv', index = False)

##### DetailBetWin

In [ ]:
query_betwin = \
"""
select UserID as transferUID, PlayDate as DateKey, count(UserID) as Counts, sum(TotalWin) as RatioTotalWin, sum(TotalBet) as RatioTotalBet
from `igs-bison-prod.Custom_Bison.DetailBetWinCustom`
where PlayDate >= '2024-08-01' and PlayDate <= '2024-09-30'
group by transferUID, DateKey
"""

job1 = client.query(query_betwin)
df = job1.to_dataframe()
# df.to_csv(r'C:\Users\chiahaowang\Documents\PythonCode\LossData\02_raw_data\07_depositRecords\depositRecords.csv', index = False)

##### DetailCoin

In [ ]:
query_coin = \
"""
select UserID as transferUID, ChangeDate as CreateDate, ChangeReason as Reason, sum(IncreaseCoins) as IncreaseCoins, sum(DecreaseCoins) as DecreaseCoins, count(UserID) as ReasonTimes
from `igs-bison-prod.ExtData_Bison.DetailCoin`
where ChangeDate >= '2024-08-01' and ChangeDate <= '2024-09-30' and CoinID = 'I000001'
group by transferUID, ChangeDate, Reason
"""

job1 = client.query(query_coin)
df = job1.to_dataframe()
# df.to_csv(r'C:\Users\chiahaowang\Documents\PythonCode\LossData\02_raw_data\07_depositRecords\depositRecords.csv', index = False)

##### GameConsume

In [ ]:
query_gameconsume = \
"""
select A.transferUID, A.CreateDate, A.CoinBefore, A.CoinAfter, B.BuyNumber
from
(
  select UserID as transferUID, ChangeDate as CreateDate, CoinBalanceAfter-IncreaseCoins as CoinBefore, CoinBalanceAfter as CoinAfter, IncreaseCoins
  from `igs-bison-prod.ExtData_Bison.DetailCoin`
  where ChangeDate >= '2024-08-01' and ChangeDate <= '2024-09-30' and CoinID = 'I000001' and ChangeReason = 'TakeDelivery'
) as A
inner join
(
  select PayDate as CreateDate, UserID as transferUID, BuyNumber, CoinAmount
  from `igs-bison-prod.ExtData_Bison.GameConsume`
  where PayDate >= '2024-08-01' and PayDate <= '2024-09-30'
) as B on A.transferUID = B.transferUID and A.CreateDate = B.CreateDate and A.IncreaseCoins = B.CoinAmount
"""

job1 = client.query(query_gameconsume)
df = job1.to_dataframe()
df.to_csv(r'C:\Users\chiahaowang\Documents\PythonCode\LossData\02_raw_data\07_depositRecords\depositRecords.csv', index = False)

##### Refund

In [6]:
query_refund = \
"""
select VoidedDate as DoneTime, UserID as transferUID, OrderID, BuyNumber, Distributor, EventTime, VoidedTime
from `igs-bison-prod.Custom_Bison.VoidedPurchase`
where ProDate >= '2024-08-01' and ProDate <= '2024-09-30'
"""

job1 = client.query(query_refund)
df = job1.to_dataframe()
df.to_csv(r'C:\Users\chiahaowang\Documents\PythonCode\LossData\02_raw_data\08_refundRecords\refundRecords.csv', index = False)